In [64]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext cython
%timeit

import numpy as np
import cython
import numpy as np
import os
import subprocess
import matplotlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The cython extension is already loaded. To reload it, use:
  %reload_ext cython


### `compute_ses`

In [58]:

def compute_ses(y, alpha):
    nobs = len(y)  

    # Forecast Array
    fh = np.full(nobs + 1, np.nan)  
    fh[0] = y[0]  
    fh[1] = y[0]  

    # Simple Exponential Smoothing
    for t in range(2, nobs + 1):
        fh[t] = alpha * y[t - 1] + (1 - alpha) * fh[t - 1]  # s[t] = alpha * y....

    return (fh[:nobs], fh[nobs])

In [59]:
np.random.seed(123)
y = np.random.random(1000)

In [60]:
time_compute_ses = %timeit -o compute_ses(y, 0.05)

598 µs ± 21.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [62]:
%%cython -a
cimport numpy as cnp
import numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cmultiplication(True)
cpdef compute_ses_cy(cython.floating[:] y, cython.floating alpha):
    cdef int n_obs = len(y)  
    cdef int t
    cdef cython.floating[:] fh = np.empty(n_obs + 1)
    cdef cython.floating alpha_y, alpha_fh

    fh[0] = y[0]
    fh[1] = y[0]
    
    for t in range(2, n_obs + 1):
        alpha_y = alpha * y[t - 1]
        alpha_fh = (1 - alpha) * fh[t - 1] 
        
        fh[t] = alpha_y + alpha_fh
        
    return (fh[:n_obs], fh[n_obs])

In [63]:
time_compute_ses_cy = %timeit -o compute_ses_cy(y, 0.05)

7.29 µs ± 231 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [68]:
res = time_compute_ses.average / time_compute_ses_cy.average
print(f'time improvement compute_ses cython: {res} X')

time improvement compute_ses cython: 82.1088993513912 X
